In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=5, edgeitems=10, linewidth=1000, suppress=True)

Using cuda device


# Carmona problem

We consider a stochastic differential game with $N$ players, and we denote by $\mathcal{I} := {1, 2, \dots, N}$ the set of players. Let $T$ be a finite time horizon. At each time $t \in [0, T]$, player $i \in \mathcal{I}$ has a state $X_{t}^{i} \in \mathbb{R}$ and takes an action $\alpha_{t}^{i} \in \mathbb{R}$. The dynamics of the controlled state process $X_{i}$ on $[0, T]$ are given by:

$$\mathrm{d} X_t^i=\left[a\left(\bar{X}_t-X_t^i\right)+\alpha_t^i\right] \mathrm{d} t+\sigma\left(\rho \mathrm{d} W_t^0+\sqrt{1-\rho^2} \mathrm{~d} W_t^i\right)$$

where $\boldsymbol{W}:=\left[W^0, W^1, \ldots, W^N\right]$ are ($N+1$) $m$-dimensional independent Brownian motions, with $W^{i}$ the individual noises and $W^{0}$ the common noise.

Given a set of strategies ($\bold{\alpha}_{t})_{t \in [0, T]}$, the cost associated to player $i$ is of the form
$$ J^{i}(\alpha): \alpha \mapsto \mathbb{E}\left[\int_0^T f^{i}\left(t, X_t, \alpha_{t}\right) dt+g^{i}(X_T)\right], $$

Here $f^i:[0, T] \times \mathbb{R}^N \times \mathbb{R}^N \rightarrow \mathbb{R}$ denotes the running cost, and $g^i: \mathbb{R}^N \rightarrow \mathbb{R}$ the terminal cost, where:
$$f^i(t, \boldsymbol{x}, \boldsymbol{\alpha})=\frac{1}{2}\left(\alpha^i\right)^2-q \alpha^i\left(\bar{x}-x^i\right)+\frac{\epsilon}{2}\left(\bar{x}-x^i\right)^2$$
$$g^i(\boldsymbol{x})=\frac{c}{2}\left(\bar{x}-x^i\right)^2$$

We solve the above problem using direct parameterization methods. For simplicity, we assume that there is no common noise, so the dynamics of $X_{i}$ on $[0, T]$ are given by:

$$\mathrm{d} X_t^i=\left[a\left(\bar{X}_t-X_t^i\right)+\alpha_t^i\right] \mathrm{d}t+\sigma\mathrm{~d} W_t^i$$

We approximate the dynamics and the expected cost by discretized versions:
$$\check{X}_{t_{n+1}}=\check{X}_{t_n} + [a\left(\check{X}_{t_n} - \check{X}^{i}_{t_n}\right) + \alpha^{i}_{t_n}] \Delta t + \sigma \Delta \check{W}_{t_n}, $$
$$\mathbb{E}\left[\sum_{n=0}^{N_T-1} f\left(t_n, \check{X}_{t_n}, \alpha_{t_n}\right) \Delta t+g\left(\check{X}_T\right)\right], $$
where $\Delta \check{W}_{t_n}=\check{W}_{t_{n+1}}-\check{W}_{t_n}$ are i.i.d random variables with distribution $\Delta\check{W} \sim \mathcal{N}\left(0, \Delta t\right)$.

We approximate the control at each time step $\alpha_{t_{n}}$ by a feedforward neural network $\alpha_{t_{n}}(.; \theta_{n})$, taking inputs $\check{X}_{t_n}$, where $\theta_{n}$ denotes all neural network's parameters at time $t_{n}$.

In [5]:
# Parameters
sigma = 0.2
q = 1
a = 1
eps = 1.5
rho = 0.2 # 0.2
c = 1
R = a**2 + 2*a*q + eps
delta_p = -(a+q) + np.sqrt(R)
delta_m = -(a+q) - np.sqrt(R)
T = 1
N_T = 5 # Number of subintervals on [0, T]
N = 2 # Number of players

We generate Brownian increments, $(\Delta\check{W}^{i}_{n})_{i=1,\dots,N, n=0,\dots,N_{T}-1}$, which are i.i.d random variables with Gaussian distribution: $\Delta\check{W} \sim \mathcal{N}\left(0, \Delta t\right)$.

We discretise the unit time interval into 100 steps, so $\Delta t = \frac{T}{N} = \frac{1}{100}$.

We include the antithetic variates to reduce the variation of our training data.

In [14]:
def BMIncrements(B, N_T, T=1):
    """Generate Brownian Motion increments

    Args:
        B: Number of sample paths
        N: Number of increments. Defaults to 100.
        T: Maximum time interval. Defaults to 1.

    Returns:
        Discretised Brownian path with antithetic variates
    """
    dat = torch.randn(B, N_T, 1)*np.sqrt(T/N_T)
    return torch.cat([dat, -dat], dim=0)

In [20]:
def one_step_simulation(x, xbar, alpha, dw):
    """Simulate one step of the dynamics of X

    Args:
        x: _description_
        m: _description_
        alpha: _description_
        dw: Brownian motion interval

    Returns:
        x_{t+1}
    """
    dt = T/N_T
    return x + (a*(xbar-x)+alpha)*dt + sigma*dw

In [39]:
def eta(t):
    numerator = -(eps-q**2)*(np.exp((delta_p-delta_m)*(T-t))-1)\
                -c*(delta_p*np.exp((delta_p - delta_m)*(T-t)) - delta_m)
    denominator = delta_m*np.exp((delta_p-delta_m)*(T-t))-delta_p \
                    -c*(np.exp((delta_p-delta_m)*(T-t)) -1)
    return numerator/denominator

In [27]:
def benchmark(w, cn, initial):
    """
    input:
        m -- tensor(batch, N+1, dim), distribution interaction
        w -- tensor(batch, N, dim), brownian increments
        cn -- tensor(batch, N+1, dim), common noise
        initial -- tensor(batch, 1, dim), starting point, has initial distribution mu_0
    return:
        X -- tensor(batch, N+1, dim), benchmark paths, no extra time dimension
    """
    dt = T/N_T
    batch, _, dim = w.size()
    
    X = torch.zeros(batch, N_T+1, dim)
    X[:, 0, :] = initial
    alpha = torch.zeros(batch, N_T, 1)
    for i in range(1, N_T+1):
        m = torch.mean(initial) + rho*sigma*cn[:, i]
        alpha = (q + eta(dt*i-dt))*(m-X[:, i-1])
        X[:, i, :] = one_step_simulation(X[:, i-1, :], m, alpha, w[:, i-1, :], cn[:, i]-cn[:, i-1])
        alpha[:, i-1] = alpha
    return X

In [29]:
# Number of sample paths to generate
B = 2**10

# Generate Brownian increments
bm = BMIncrements(B//2, N_T, T=1)

# Generate common noise, w0
w_cn = BMIncrements(B//2, N_T, T=1)
w0 = torch.zeros(B, N_T+1, 1)
for i in range(1, N_T+1):
    w0[:, i] = w0[:, i-1] + w_cn[:, i-1]
bms = torch.utils.data.TensorDataset(bm, w0)
bmDataLoader = torch.utils.data.DataLoader(bms, batch_size=2**7)

alpha_0 = 0 # Initial guess of optimal strategies
initial = torch.zeros(B, 1) # Initial, X0

In [30]:
initial

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [1]:
def f(x, xbar, alpha):
    """The running cost at time t

    Args:
        x: State at time t
        xbar: Mean at time t
        alpha: Control at time t
    """
    return 0.5*alpha**2 - q*alpha*(xbar-x) + 0.5*eps*(xbar-x)**2

def g(x, xbar):
    """Final time penalty

    Args:
        x: State of player i at time T
        xbar: Mean at time T
    """
    return c/2*(xbar-x)**2

In [ ]:
def loss(alpha, ):


Plot sample paths:

# Neural Network

We use a recurrent neural network

In [ ]:
# Neural network architecture parameters
num_input_nodes = I
num_output_nodes = I

In [ ]:
# Define neural networks for each player
model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='ReLU', input_shape=(num_input_nodes,), kernel_initializer=tf.keras.initializers.GlorotNormal),
    tf.keras.layers.Dense(64, activation='ReLU'),
    tf.keras.layers.Dense(num_output_nodes)
])

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='ReLU', input_shape=(num_input_nodes,), kernel_initializer=tf.keras.initializers.GlorotNormal),
    tf.keras.layers.Dense(64, activation='ReLU'),
    tf.keras.layers.Dense(num_output_nodes)
])

In [ ]:
# Hyper-parameters
batch_size = 64
lr = 1e-3

# Adam optimizer
mse = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.experimental.Adam(learning_rate=lr, amsgrad=True, jit_compile=True)

model.compile(
    optimizer,
    loss='mse'
)